In [1]:
# import libraries

from pandas_datareader import data as pdr
from yahoo_finance import Share as sshare
from yahoo_fin import stock_info as si 
from pandas import ExcelWriter
import yfinance as yf
import pandas as pd
import datetime
import time

In [2]:
yf.pdr_override()

In [3]:
# Variables
tickers = si.tickers_sp500()

C:\Users\caro_\anaconda3\envs\scanyahoo\lib\site-packages\yahoo_fin\stock_info.py:130: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  sp500["Symbol"] = sp500["Symbol"].str.replace(".", "-")


In [4]:
tickers = [item.replace(".", "-") for item in tickers]
index_name = '^GSPC'

In [5]:
start_date = datetime.datetime.now() - datetime.timedelta(days=365) # this is to setup start time based on a delta rather than fixed time
start_date

datetime.datetime(2020, 4, 9, 18, 48, 9, 175888)

In [6]:
end_date =datetime.date.today()
end_date

datetime.date(2021, 4, 9)

In [7]:
export_list = pd.DataFrame(columns=['stock', 'rs_rating', '50d SMA', '150d SMA', '200d SMA', '52wlow', '52whigh'])
export_list

,stock,rs_rating,50d SMA,150d SMA,200d SMA,52wlow,52whigh


In [8]:
returns_multiples =[]


In [9]:
# Calculate Index Returns

index_df = pdr.get_data_yahoo(index_name, start_date, end_date)
index_df['perc_change'] = index_df['Adj Close'].pct_change()
index_df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,perc_change
Date,,,,,,,
2020-04-09,2776.989990,2818.570068,2762.360107,2789.820068,2789.820068,7880140000,NaN
2020-04-13,2782.459961,2782.459961,2721.169922,2761.629883,2761.629883,5274310000,-0.010105
2020-04-14,2805.100098,2851.850098,2805.100098,2846.060059,2846.060059,5567400000,0.030573
2020-04-15,2795.639893,2801.879883,2761.540039,2783.360107,2783.360107,5203390000,-0.022030
2020-04-16,2799.340088,2806.510010,2764.320068,2799.550049,2799.550049,5179990000,0.005817


In [10]:
index_return = (index_df['perc_change'] + 1).cumprod()[-1]
index_return

1.4686143985925397

In [19]:
# delete "FOX" from the dataframe to avoid errors
select_fox = tickers.index('FOX')
print(select_fox)

188


In [20]:
tickers.pop(188)

'FOX'

In [21]:
# Find the top 30% performing stocks (relative to the S&P 500)

for ticker in tickers:
    # Download historical data as a csv file for each stock
    df = pdr.get_data_yahoo(ticker, start_date, end_date)
    df.to_csv(f'csv_files/{ticker}.csv')

    # Calculating returns relative to the market 
    df['perc_change'] = df['Adj Close'].pct_change()
    stock_return = (df['perc_change'] + 1).cumprod()[-1]

    returns_multiple = round((stock_return/ index_return), 2)
    returns_multiples.extend([returns_multiple])

    print (f' Ticker: {ticker}; Returns Multiple against S&P 500: {returns_multiple}\n')
    time.sleep(1)



ple against S&P 500: 1.02

[*********************100%***********************]  1 of 1 completed
 Ticker: NI; Returns Multiple against S&P 500: 0.65

[*********************100%***********************]  1 of 1 completed
 Ticker: NKE; Returns Multiple against S&P 500: 1.06

[*********************100%***********************]  1 of 1 completed
 Ticker: NLOK; Returns Multiple against S&P 500: 0.8

[*********************100%***********************]  1 of 1 completed
 Ticker: NLSN; Returns Multiple against S&P 500: 1.25

[*********************100%***********************]  1 of 1 completed
 Ticker: NOC; Returns Multiple against S&P 500: 0.69

[*********************100%***********************]  1 of 1 completed
 Ticker: NOV; Returns Multiple against S&P 500: 0.79

[*********************100%***********************]  1 of 1 completed
 Ticker: NOW; Returns Multiple against S&P 500: 1.28

[*********************100%***********************]  1 of 1 completed
 Ticker: NRG; Returns Multiple against S&P 

IndexError: index -1 is out of bounds for axis 0 with size 0

In [22]:
#fox = pdr.get_data_yahoo("FOX", start_date, end_date)

In [23]:
#print(fox)

In [24]:
#fox.head()

In [25]:
# Creating DataFrame with only top 30% of stocks

rs_df = pd.DataFrame(list(zip(tickers, returns_multiples)), columns=['ticker','returns_multiple'])

In [26]:
rs_df['rs_rating'] = rs_df.returns_multiple.rank(pct=True)*100

In [27]:
rs_df.head()

,ticker,returns_multiple,rs_rating
0,A,1.14,63.392857
1,AAL,1.29,80.257937
2,AAP,1.16,67.757937
3,AAPL,1.34,83.333333
4,ABBV,0.95,41.865079


In [28]:
rs_df_top_30 = rs_df[rs_df.rs_rating >= rs_df.rs_rating.quantile(0.70)]
rs_df_top_30.head()

,ticker,returns_multiple,rs_rating
1,AAL,1.29,80.257937
3,AAPL,1.34,83.333333
6,ABMD,1.40,87.103175
13,ADSK,1.21,72.817460
16,AES,1.32,82.142857


In [29]:
rs_df_top_30.count()

ticker              154
returns_multiple    154
rs_rating           154
dtype: int64

In [34]:
rs_stocks = rs_df_top_30['ticker']
rs_stocks.head()

1      AAL
3     AAPL
6     ABMD
13    ADSK
16     AES
Name: ticker, dtype: object

In [54]:
for stock in rs_stocks:
    try:
        df = pd.read_csv(f'csv_files/{stock}.csv', index_col=0)
        sma = [50, 150, 200]
        for x in sma:
            df["SMA_"+str(x)] = round(df['Adj Close'].rolling(window=x).mean(),2)
        
        # Storing requiredvalues
        currentClose =df["Adj Close"][-1]
        moving_average_50 = df["SMA_50"][-1]
        moving_average_150 = df["SMA_150"][-1]
        moving_average_200 = df["SMA_200"][-1]
        low_of_52week = round(min(df["Low"][-260:]), 2)
        high_of_52week = round(max(df["High"][-260:]), 2)
        print(low_of_52week)
        rs_rating = round(rs_df[rs_df['ticker']==stock].rs_rating.tolist()[0])

        try:
            moving_average_200_20 = df["SMA_200"][-20]
        
        except Exception:
            moving_average_200_20 = 0

        # Condition 1: Current Price > 150 SMA and 200 SMA
        condition_1 = (currentClose > moving_average_150) and (currentClose > moving_average_200)

        # Condition 2: 150 SMA > 200 SMA
        condition_2 = moving_average_150 > moving_average_200

        # Condition 3: 200 SMA trending up for at least 1 month
        condition_3 = moving_average_200 > moving_average_200_20

        # Condition 4: 50 SMA > 150 SMA and 50 SMA > 200 SMA
        condition_4 = (moving_average_50 > moving_average_150) and (moving_average_50> moving_average_200)

        # Condition 5: Current Price > 50 SMA
        condition_5 = currentClose > moving_average_50

        # Condition 6: Current Price is at least 30% above 52 week low
        condition_6 = currentClose >=(1.3*low_of_52week)

        # Condition 7: Current Price is within 25% of 52 week high
        condition_7 = currentClose >=(0.75*high_of_52week)

        # If all the above conditions are true then add stock to export_list columns: 'stock', 'rs_rating', '50d SMA', '150d SMA', '200d SMA', '52wlow', '52whigh']
        if(condition_1 and condition_2 and condition_3 and condition_4 and condition_5 and condition_6 and condition_7):
           export_list = export_list.append({'stock': stock, "rs_rating": rs_rating, "50d SMA": moving_average_50, "150d SMA": moving_average_150, "200d SMA": moving_average_200, "52wlow": low_of_52week, "52whigh": high_of_52week }, ignore_index=True) 
           print(stock + " made the Minervini requirements")


    except Exception as e:
        print (e)
        print(f"could not gather data on {stock}")



8.25
AAL made the Minervini requirements
66.18
AAPL made the Minervini requirements
152.21
ABMD made the Minervini requirements
156.91
ADSK made the Minervini requirements
11.0
AES made the Minervini requirements
22.14
AIG made the Minervini requirements
55.75
178.7
ALGN made the Minervini requirements
23.39
ALK made the Minervini requirements
46.22
AMAT made the Minervini requirements
100.11
AMP made the Minervini requirements
7.08
56.04
246.8
AVGO made the Minervini requirements
63.59
BBY made the Minervini requirements
14.91
BEN made the Minervini requirements
440.0
BLK made the Minervini requirements
24.27
BWA made the Minervini requirements
12.26
CARR made the Minervini requirements
100.22
CAT made the Minervini requirements
33.03
CBRE made the Minervini requirements
10.95
CCL made the Minervini requirements
70.4
CDNS made the Minervini requirements
69.69
CE made the Minervini requirements
17.15
CFG made the Minervini requirements
25.8
CMA made the Minervini requirements
728.0
CMG

In [55]:
export_list_final = export_list.sort_values(by='rs_rating', ascending=False)
print('\n', export_list_final)


     stock rs_rating  50d SMA  150d SMA  200d SMA  52wlow  52whigh
118   XYL       100   100.69     95.22     89.49   56.63   108.84
74    MOS       100    30.27     24.01     21.73    9.57    35.20
27    CZR       100    86.68     71.12     63.22   14.62   106.20
90     PH        99   294.17    260.23    242.39  123.32   323.80
87      O        99    61.75     60.50     59.93   47.68    66.80
..    ...       ...      ...       ...       ...     ...      ...
17   CBRE        70    74.14     61.88     57.57   33.03    82.05
13    BLK        70   724.12    673.56    646.03  440.00   802.01
108   UNH        70   345.57    335.29    327.13  258.18   380.50
66      L        70    49.80     43.38     41.45   27.33    53.43
104  TSCO        70   163.33    148.55    146.89   86.43   180.85

[122 rows x 7 columns]


In [62]:
#! pip install openpyxl
import openpyxl
writer = ExcelWriter("ScreenOutput.xlsx")
export_list_final.to_excel(writer, "Sheet1")
writer.save()